# Stereo Robot Navigation
## Federico Calzoni, Lorenza Guerriero, Arka Patra

### Introduction
The ability of an autonomous vehicle to sense obstacles and navigate its surroundings is a key role in the development of modern robots. For this reason we present a project in which a robot, through the principle of a stereo camera, is able to perceive the obstacles in front of his view. Such technology could be implemented by a navigation system for autonomous vehicles to automatically avoid obstacles.

### Objective
The objective of the project aims to use synchronized video sequences captured by a stereo camera mounted on a moving vehicle to detect spatial information of the frontal environment. The primary task is to develop an area-based stereo matching algorithm capable of generating a dense disparity map from the syncronized stereo camera inputs (robotL.avi for the left view, robotR.avi for the right view). Then by using geometrical calculations an extimated distance from the robot to the central area of the view need to be calculated. Additionally, an alarm is triggered whenever the distance falls below 0.8 meters.
Finally, given that in the center of the sciene there is a chessboard. The dimension of it (height and width) must be computed. This will be useful in order to compare it to the real values and give an extimation of the accuracy of the method used.  


#Todo explain better (Functional Specifications)

The system follows a series of steps to achieve its objectives:

1) Disparity Map Computation\
Compute the disparity map in a central area of the reference frame to sense distances in the portion of the environment relevant to the vehicle's trajectory.

2) Main Disparity Estimation\
Estimate the main disparity for the frontal portion of the environment based on the disparity map of the central area of the reference frame.

3) Distance Calculation\
Determine the distance of the obstacle from the moving vehicle based on the main disparities estimated from each pair of frames.

4) Output Generation and Alarm Trigger\
Generate output conveying distance information to the user and trigger an alarm when the distance falls below a predefined threshold.

5) Estimate the size of the board\
Use the found board corners to estimate the size of the board. Then compare it with the real size of the known one (125mm x 178mm)


### Stereo Matching Algorithm

The stereo matching algorithm computes the disparity map for each pair of synchronized frames. The algorithm utilizes the Sum of Absolute Differences (SAD) dissimilarity measure to compare intensities between corresponding points in the left and right images.

# todo explain SAD

### Distance Estimation

Based on the computed disparity map, the system estimates the main disparity for the frontal portion of the environment. Using the provided parameters (focal length and baseline), it calculates the distance (in mm) of obstacles from the moving vehicle.

# todo:explain formula


# explain code (Implementation Details)

Algorithm Development:
The stereo matching algorithm is developed based on the SAD dissimilarity measure. It compares intensities between corresponding points in rectified images to compute the disparity map.

2 Parameter Selection
The size of the window for matching and the fixed disparity range. 

3 Output Generation
The system generates output in each pair of frames, conveying distance information from the camera to the obstacle. An alarm is triggered when the distance goes below 0.8 meters. 

# todo (Results)

# todo (Conclusions and enhancements that can be done)


# References
# todo